In [16]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split
import random as rand
from random import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [10]:
df = pd.read_excel('aug_nine_var.xlsx')
X = df.iloc[:,1:22]
y = df.iloc[:,22:23].to_numpy()

In [11]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42, shuffle=False)
np.random.shuffle(X_train)
np.random.shuffle(y_train)

In [15]:
episode = 50
eps_start = 0.9
eps_end = 0.05
eps_decay = 200

gamma = 0.8 #감마는 할인계수, 에이전트가 현재를 미래보다 더 가치있게 여기는 것

lr = 0.001
batch_size = 64

In [ ]:
class Agent:
    def __intit__(self):
        self.model = tf.keras.models.load_model('val_0.06_dnn.h5')
        self.steps_done = 0
        self.memory = deque(maxlen=1000)
    
    def memorize(self, state, action, reward, next_state):
        self.memory.append(
            state, action, reward, next_state
        )
    
    def act(self, state):
        eps_threshold = eps_end + (eps_start - eps_end) * math.exp(-1. *self.steps_done / eps_decay)
        self.steps_done = self.steps_done + 1

        

In [14]:
class enviornment():
    def __init__(self):
        self.last_state = 0
    def reset(self):
        self.state = 0
        return self.state
    def sample(self):
        self.last_action = randint(0,41)
        return self.last_action
    def step(self,action):
        global result_board
        global Q
        one  = (result_board)
        if(action < 21):
            result_board[action] -= 0.01
            r = action

        else:
            result_board[action-21] += 0.01
            r = action-21
        if(result_board[r] < 0 or result_board[r] > 1):
            self.last_state = self.state
        if(action < 21):
            result_board[action] += 0.01
        else:
            result_board[action-21] -= 0.01

        next = result_board
        n = False
        rt = 0
        global result_boards

        for tt in range(0,len(result_boards)):
            c = result_boards[tt]
            c = list(np.round(c,6))
            next = list(np.round(next,6))

            if(list(c) == list(next)):
                n = True
                break
        if(n):
            n = False
            if(rt == 1):
                self.last_state = self.state
                rt == 0
      # print("rebuilding..! state: %d - %d" % (result_boards.index(c),self.state + 1))
            next_state = tt
        else:
            rt = 1
            result_boards.append(next)
            Q = np.concatenate([Q,np.zeros((1,42))],axis = 0)
            next_state = self.state + 1
            self.state += 1
            self.last_state = self.state

        c = evaluate_loss(next)
        return next_state,c,False,None
  
    def read(self,next_state):
        self.state = next_state
#evaluate_loss의 경우 공식을 적용후 결과를 내뱉는 함수.
    def evaluate_loss(y_pred):
        global max_loss
        global model
        global X_test
        global y_test
        y_pred = np.array(y_pred).reshape((1,21,))
        last_data = X_test[-1]
        pred_1 = model.predict(y_pred,verbose=0)
        b = np.sqrt(np.sum(np.square(y_pred-last_data)))
        a = (np.square(y_test[-1]-pred_1))
        return max_loss-1*(pred_1*10+a+(np.abs(b*10-a)))
#result_board 초기화'
global max_loss
max_loss = 282439870
global result_board
result_board = X_test[-1]
#result_board의 리스트 초기화
#state 관리용
global result_boards
result_boards = []
result_boards.append(list(result_board))
env = enviornment()
STATES = 2
ACTIONS = 42

env.reset()
#Q가 바로 action에 대한 reward를 지정해주는 array이다.
global Q
Q = np.zeros((STATES, ACTIONS))
EPISODES = 2000 # how many times to run the enviornment from the beginning
MAX_STEPS = 100  # max number of steps allowed for each run of enviornment
LEARNING_RATE = 0.01  # learning rate
GAMMA = 0.96
min_thing = 0.1
RENDER = False # if you want to see training set to true
result_ = []
epsilon = 1
y = []
rate = 0.9995
rewards = []
for episode in range(EPISODES):
    y_result = model.predict(np.array(result_board).reshape(1,21,),verbose=0)[0][0]
    print("loading... try: %d epsilon: %.4f y_result: %.4f" % (episode,epsilon,y_result))
    print((scaler.inverse_transform(np.array(result_board).reshape(1,21,)))[0])
    state = env.reset()
    result_board = X_test[-1]
    for _ in range(MAX_STEPS):
        if rand.random() < epsilon:
            action = env.sample()  
        else:
            action = np.argmax(Q[state, :])
            next_state, reward, done, _ = env.step(action)
        if(Q[state,action] == 0):
            Q[state, action] = LEARNING_RATE * (reward + GAMMA * np.max(Q[next_state, :]) - Q[state, action])
        else:
            Q[state, action] = Q[state, action] + LEARNING_RATE * (reward + GAMMA * np.max(Q[next_state, :]) - Q[state, action])
            state = next_state
            env.read(next_state)
        if(state % 1000 == 0):
            print("processing... state : %d" % (state))
            state += 1
    if(episode % 1000 == 0):
        print("learning... episode : %d" % (episode))
        episode += 1
    rewards.append(reward)
    epsilon *= rate
    epsilon = max(epsilon,min_thing)
    k = (scaler.inverse_transform(np.array(result_board).reshape(1,21,)))[0]
    k = k.tolist()
    y_result = model.predict(np.array(result_board).reshape(1,21,),verbose=0)[0][0]
    k.append(y_result)
    result_.append(k)
    result_ = np.array(result_)
    pl = pd.DataFrame(result_)
    pl.to_excel("result.xlsx")
    result_ = list(result_)
    y = list(y)
    print("Average reward: : %d (%.4f)" % (list(np.array(sum(rewards)/len(rewards)))[0][0],list(np.array(sum(rewards)/len(rewards)))[0][0]-max_loss))
# and now we can see our Q values!

def get_average(values):
    return sum(values)/len(values)

avg_rewards = []
for i in range(0, len(rewards), 100):
    avg_rewards.append(get_average(rewards)) 
plt.plot(avg_rewards)
plt.ylabel('average reward')
plt.xlabel('episodes (100\'s)')
plt.show()

loading... try: 0 epsilon: 1.0000 y_result: 40.5117
[ 9.7700000e-01  5.1824477e+07  9.7740880e+06  1.3061074e+07
  2.9538440e+06  2.5789006e+07  2.5301000e+04  2.4200000e+04
  2.2801000e+04  1.0087000e+04  2.0000000e+00 -4.8000000e+00
 -5.0000000e-01  0.0000000e+00 -1.0000000e-01 -1.0000000e-01
  9.9330000e+01  1.7500000e+00  1.1285800e+03  5.0930000e+01
  1.8981926e+06]


NameError: name 'reward' is not defined